In [104]:
class StartPoint:
    def __init__(self, x=None, y=None):        
        self.x = None
        self.y = None
        if x is not None:
            self.x = x
        else:
            self.x = 10 #int(random.random() * 200)
        if y is not None:
            self.y = y
        else:
            self.y = 10 #int(random.random() * 200)
   
    def getX(self):
        return self.x
   
    def getY(self):
        return self.y




In [116]:
import numpy as np

class Individ:
    def __init__(self, step, size, start_point, end_point, our_field):
        self.route = np.random.randint(-1,2, size=(2,size))*step  #   step*(-1) / step*1 / step*0
        self.individ_size = size
        self.fitness = 0
        self.setfitness(start_point, end_point, our_field)
        
    def setfitness(self, start_point, end_point, our_field):   #start_point = (x,y)
        cumsum = np.cumsum(self.route, axis=1) + np.array(start_point).reshape((2,1))
        index = self.checkblocks(cumsum, our_field)
        #print ('cumsum ', cumsum[:, index])
        distance = np.linalg.norm(cumsum[:, index] - np.array(end_point))      
        self.fitness = distance    # check normalization   
        #print('fitness individ ', self.fitness)
        
    def getfitness(self):
        return self.fitness
    
    def checkblocks(self, cumsum, our_field):
        for i in range(1, self.individ_size):
            if (any(cumsum[:, i] < 0) or (cumsum[0, i] > our_field.field_size[0]) or (cumsum[1, i] > our_field.field_size[1])):
                return i-1 
            elif our_field.getvalue(cumsum[0, i], cumsum[1, i]):
                return i-1
        return self.individ_size-1
    
        

In [106]:
class Field:
    def __init__(self, size):
        self.matrix = np.zeros(size)
        self.matrix[0, :] = 1
        self.matrix[:, 0] = 1
        self.matrix[:, size[1]-1] = 1
        self.matrix[size[0]-1, 0] = 1
        self.field_size = size
    
    def addSquareBlock(self, left_top, right_bottom):  #point = (x,y)
        for i in range(left_top[0], right_bottom[0]):
            for j in range(left_top[1], right_bottom[1]):
                self.matrix[i, j] = 1
    
    # add diff shapes of blocks
        
    def getmatrix(self):
        return self.matrix
    
    def getvalue(self,x,y):
        return self.matrix[x,y]
    
        
        

In [107]:
class Population:
    def __init__(self, step, individSize, populationSize, start_point, end_point, our_field):
        self.population = []
        self.fitness = []
        for i in range(0, populationSize):
            newInd = Individ(step, individSize, start_point, end_point, our_field)
            self.population.append(newInd)            
            self.fitness.append(newInd.fitness)
                
    def getFitness(self):
        return self.fitness
    
    def getBest(self):
        #print('fitness ', self.fitness)
        #print('population ', self.population)
        sortedPopulation = [pop for _,pop in sorted(zip(self.fitness, self.population), key = lambda x: x[0])]
        
        #self.fitness.sort(reverse=True)
        
        
        return sortedPopulation[0]
    
    def add_individ(self, newIndivid):
        self.population.append(newIndivid)
        self.fitness.append(newIndivid.fitness)
    
        
    
    

In [121]:
import random

class GA:
    def __init__(self): #, tourmanager):
        #self.tourmanager = tourmanager
        self.mutationRate = 0.5
        self.tournamentSize = 10
        self. crossoverProbability = 0.8
        self.elitism = True
        
    def chooseParents(self, newPopulation):
        #print ('parent fitness ', newPopulation.fitness)
        reversed_fitness = 1.0/np.array(newPopulation.fitness)
        select_prob = reversed_fitness/sum(reversed_fitness)
        #add tournamentsize
        selected = np.random.choice(newPopulation.population, 2, p=select_prob, replace=False)
        return list(selected)
    
    def crossover(self, parents):
        startPos = random.randint(0,parents[0].individ_size)
        endPos = random.randint(startPos, parents[0].individ_size)   
        pp = random.random()
        if pp <= self.crossoverProbability:
            
            parents[0].route[:, startPos:endPos], parents[1].route[:, startPos:endPos] = parents[1].route[:, startPos:endPos], parents[0].route[:, startPos:endPos]
            
        return parents
    
    def mutate(self, children):
        if random.random() < self.mutationRate:
            indx = random.randint(0, children[0].individ_size-1)
            children[0].route[0, indx], children[0].route[1, indx] = children[0].route[1, indx], children[0].route[0, indx]
        if random.random() < self.mutationRate:
            indx = random.randint(0, children[1].individ_size-1)
            children[1].route[0, indx], children[1].route[1, indx] = children[1].route[1, indx], children[1].route[0, indx]
        return children
    
    def fittest(self, child_list, n): # n - how much return
        child_list.sort(key = lambda x: x.fitness)
        return child_list[:n]
              
        
    def evolve(self, step, individSize, populationSize, generation, start_point, end_point, our_field):  #generation - previous population
        
        children_list = []
        for i in range(populationSize):
            #choose parents
            #print('generation fitness ', generation.fitness)
            parents = self.chooseParents(generation)  
            #make 2 children
            children = self.crossover(parents)
            #mutate
            children = self.mutate(children)
            children[0].setfitness(start_point, end_point, our_field)
            children[1].setfitness(start_point, end_point, our_field)
            #print('children list ', children_list)
            #print('children ', type(children), len(children))
            children_list = children_list + children
            
        #Create new generation with elitism?
        
        newPopulation = Population(step, individSize, populationSize, start_point, end_point, our_field)
        if self.elitism == True:
            newPopulation.add_individ(generation.getBest())
            best_children = self.fittest(children_list, populationSize-1)
        else:
            best_children = self.fittest(children_list, populationSize)
        
        for child in best_children:
            newPopulation.add_individ(child)
            
        #print('new population ', newPopulation)
                  
        return newPopulation
        

In [122]:

gameField = Field(size=(30, 30))
initialPop = Population(step=2, individSize=10, populationSize=10, start_point=(0,0), end_point=(20,20), our_field=gameField)
ga = GA()
print('evolve function: ')

while not any(np.array(initialPop.fitness) < 8):
    initialPop = ga.evolve(step = 2, individSize = 10, populationSize = 10, generation = initialPop, start_point=(0,0), end_point=(20,20), our_field=gameField)
    print('min fitness ', min(initialPop.fitness))



evolve function: 
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  18.8679622641
min fitness  18.8679622641
min fitness  19.6977156036
min fitness  20.591260282
min fitness  22.803508502
min fitness  22.803508502
min fitness  22.627416998
min fitness  20.591260282
min fitness  20.591260282
min fitness  20.591260282
min fitness  21.2602916255
min fitness  21.2602916255
min fitness  21.6333076528
min fitness  21.2602916255
min fitness  21.6333076528
min fitness  18.4390889146
min fitness  18.8679622641
min fitness  18.8679622641
min fitness  21.2602916255
min fitness  17.0880074906
min fitness  17.0880074906
min fitness  17.0880074906
min fitness  24.0831891576
min fitness  15.2315462117
min fitness  21.6333076528
min fitness  19.7989898732
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0

min fitness  17.2046505341
min fitness  12.8062484749
min fitness  18.8679622641
min fitness  18.8679622641
min fitness  18.4390889146
min fitness  21.6333076528
min fitness  22.803508502
min fitness  14.1421356237
min fitness  14.4222051019
min fitness  25.4558441227
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  22.803508502
min fitness  20.591260282
min fitness  20.0
min fitness  25.4558441227
min fitness  22.803508502
min fitness  18.4390889146
min fitness  15.6204993518
min fitness  13.416407865
min fitness  22.803508502
min fitness  21.6333076528
min fitness  21.2602916255
min fitness  21.6333076528
min fitness  22.803508502
min fitness  22.803508502
min fitness  22.803508502
min fitness  22.803508502
min fitness  25.4558441227
min fitness  24.0831891576
min fitness  17.2046505341
min fitness  16.1245154966
min fitness  15.6204993518
min fitness  15.6204993518
min fitness  21.6333076528
min fitness  21.6333

min fitness  25.4558441227
min fitness  24.0831891576
min fitness  18.4390889146
min fitness  18.8679622641
min fitness  19.6977156036
min fitness  18.8679622641
min fitness  19.6977156036
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  18.8679622641
min fitness  18.8679622641
min fitness  17.88854382
min fitness  18.8679622641
min fitness  20.0
min fitness  22.803508502
min fitness  25.4558441227
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  22.803508502
min fitness  22.803508502
min fitness  22.803508502
min fitness  17.2046505341
min fitness  24.0831891576
min fitness  21.6333076528
min fitness  21.6333076528
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  22.803508502
min fitness  22.803508502
min fitness  20.0
min fitness  22.803508502
min fitness  16.9705627485
min fitness  21.6333076528
min fitness  20.0
min fitness  20.0
min fitness  13.416407865
min fitness  19.7989

min fitness  22.803508502
min fitness  16.9705627485
min fitness  15.6204993518
min fitness  16.1245154966
min fitness  14.4222051019
min fitness  14.4222051019
min fitness  14.4222051019
min fitness  14.4222051019
min fitness  10.7703296143
min fitness  10.0
min fitness  24.0831891576
min fitness  16.1245154966
min fitness  18.4390889146
min fitness  18.4390889146
min fitness  22.803508502
min fitness  15.6204993518
min fitness  15.6204993518
min fitness  14.1421356237
min fitness  14.1421356237
min fitness  21.6333076528
min fitness  21.6333076528
min fitness  18.4390889146
min fitness  15.6204993518
min fitness  10.7703296143
min fitness  21.6333076528
min fitness  21.6333076528
min fitness  21.6333076528
min fitness  21.2602916255
min fitness  21.6333076528
min fitness  16.9705627485
min fitness  18.973665961
min fitness  21.2602916255
min fitness  20.0
min fitness  19.7989898732
min fitness  18.8679622641
min fitness  17.88854382
min fitness  16.9705627485
min fitness  17.20465053

min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  18.4390889146
min fitness  18.4390889146
min fitness  20.0
min fitness  18.8679622641
min fitness  21.6333076528
min fitness  18.8679622641
min fitness  22.803508502
min fitness  22.803508502
min fitness  24.0831891576
min fitness  22.803508502
min fitness  22.803508502
min fitness  18.4390889146
min fitness  15.6204993518
min fitness  15.6204993518
min fitness  15.6204993518
min fitness  16.1245154966
min fitness  15.6204993518
min fitness  14.4222051019
min fitness  16.1245154966
min fitness  18.4390889146
min fitness  24.0831891576
min fitness  22.803508502
min fitness  22.803508502
min fitness  22.803508502
min fitness  21.6333076528
min fitness  20.591260282
min fitness  20.591260282
min fitness  20.591260282
min fitness  19.6977156036
min fitness  18.8679622641
min fitness  21.6333076528
min fitness  25.4558441227
min fitness  24.0831891576
min fitness  25.4558441227
min fitness  24.0831

min fitness  22.803508502
min fitness  25.4558441227
min fitness  25.4558441227
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  21.2602916255
min fitness  22.803508502
min fitness  24.0831891576
min fitness  21.6333076528
min fitness  18.4390889146
min fitness  15.2315462117
min fitness  21.2602916255
min fitness  15.2315462117
min fitness  19.7989898732
min fitness  19.7989898732
min fitness  20.0
min fitness  18.8679622641
min fitness  17.2046505341
min fitness  17.2046505341
min fitness  17.2046505341
min fitness  17.2046505341
min fitness  15.6204993518
min fitness  17.2046505341
min fitness  17.2046505341
min fitness  20.591260282
min fitness  20.591260282
min fitness  20.591260282
min fitness  20.591260282
min fitness  18.4390889146
min fitness  16.1245154966
min fitness  16.1245154966
min fitness  15.6204993518
min fitness  21.6333076528
min fitness  22.627416998
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0

min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  21.6333076528
min fitness  17.2046505341
min fitness  20.0
min fitness  16.9705627485
min fitness  16.9705627485
min fitness  16.9705627485
min fitness  16.9705627485
min fitness  17.2046505341
min fitness  20.0
min fitness  24.0831891576
min fitness  22.803508502
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  21.6333076528
min fitness  21.6333076528
min fitness  24.0831891576
min fitness  21.2602916255
min fitness  18.4390889146
min fitness  16.9705627485
min fitness  16.9705627485
min fitness  16.9705627485
min fitness  16.9705627485
min fitness  20.0
min fitness  22.803508502
min fitness  20.591260282
min fitness  20.591260282
min fitness  19.6977156036
min fitness  18.8679622641
min fitness  15.6204993518
min fitness  17.88854382
min fitness  18.4390889146
min fitness  22.803508502
min fitness  22.803508502
min fitness  17.2046505341
min fitne

min fitness  18.4390889146
min fitness  22.803508502
min fitness  21.2602916255
min fitness  19.7989898732
min fitness  20.0
min fitness  20.0
min fitness  12.8062484749
min fitness  17.2046505341
min fitness  19.6977156036
min fitness  20.0
min fitness  21.2602916255
min fitness  20.0
min fitness  22.803508502
min fitness  22.803508502
min fitness  21.6333076528
min fitness  22.803508502
min fitness  20.0
min fitness  20.0
min fitness  20.591260282
min fitness  17.2046505341
min fitness  15.6204993518
min fitness  25.4558441227
min fitness  25.4558441227
min fitness  24.0831891576
min fitness  24.0831891576
min fitness  20.591260282
min fitness  22.803508502
min fitness  22.803508502
min fitness  22.803508502
min fitness  17.2046505341
min fitness  17.2046505341
min fitness  17.2046505341
min fitness  24.0831891576
min fitness  18.4390889146
min fitness  18.4390889146
min fitness  16.9705627485
min fitness  16.9705627485
min fitness  16.9705627485
min fitness  17.88854382
min fitness 

KeyboardInterrupt: 